# pattern in text 筆記

## 緣起
想確認 python 字串匹配演算法底層實作


## 題目描述

給定兩個字串 `text` (文本) 和 `pattern` (模式)。  
請你在 `text` 字串中找出 `pattern` 字串**所有**出現的索引 (0-based)，並將它們以一個 list/array 的形式回傳。  

* 如果 `pattern` 不是 `text` 的一部分，則回傳一個空 list `[]`。
* 匹配可以是**重疊 (overlapping)** 的。

#### draft, unfinished

### 範例 1:

**Input:** `text = "sadbutsad"`, `pattern = "sad"`
**Output:** `[0, 6]`
**Explanation:** "sad" 在索引 0 和 6 出現。

### 範例 2:

**Input:** `text = "leetcode"`, `pattern = "leeto"`
**Output:** `[]`
**Explanation:** "leeto" 未在 "leetcode" 中出現。

### 範例 3 (重疊匹配):

**Input:** `text = "aaaaa"`, `pattern = "aa"`
**Output:** `[0, 1, 2, 3]`
**Explanation:** "aa" 分別在索引 0, 1, 2, 3 開始。

### 範例 4 (空 Pattern):

**Input:** `text = "abc"`, `pattern = ""`
**Output:** `[0, 1, 2, 3]`
**Explanation:** 空字串 `""` 在 `text` 的*每一個*位置 (包含結尾) 都是一個匹配。

In [8]:

test_cases = [
    ("sadbutsad", "sad", [0, 6]),        # 範例 1
    ("leetcode", "leeto", []),           # 範例 2
    ("aaaaa", "aa", [0, 1, 2, 3]),       # 範例 3 (重疊)
    ("abc", "", [0, 1, 2, 3]),          # 範例 4 (空 pattern)
]

In [ ]:
def run_test_suite(algo_func, algo_name: str, test_cases: list):
    """
    Encapsulated test runner.
    
    Args:
        algo_func: The algorithm function to test (e.g., str_naive).
        algo_name: The display name of the algorithm (e.g., "Naive").
        test_cases: The list of test cases to run.
    """
    print(f"Testing {algo_name}...")
    all_passed = True
    
    for i, (text, pattern, expected) in enumerate(test_cases):
        try:
            # Call the specific algorithm function passed in
            result = algo_func(text, pattern)
            
            # Validate the result
            if result == expected:
                print(f"  [PASS] Case {i}: ({text!r}, {pattern!r}) -> {result}")
            else:
                print(f"  [FAIL] Case {i}: ({text!r}, {pattern!r})\n"
                      f"         Expected: {expected}, Got: {result}")
                all_passed = False
                
        except Exception as e:
            print(f"  [ERROR] Case {i}: ({text!r}, {pattern!r}) - {e}")
            all_passed = False

    print("-" * 30)
    if all_passed:
        print(f"✅ {algo_name}: All test cases passed!")
    else:
        print(f"❌ {algo_name}: Some test cases failed.")
    
    print("\n") # Add a newline for separation
    return all_passed

: 

### 1. Naive (樸素算法)

我們假設 `text` 的長度為 $N$，`pattern` 的長度為 $M$。

* **時間複雜度: $O(N \cdot M)$**
* **空間複雜度: $O(k)$ 或 $O(N)$**

In [ ]:
def str_naive(text: str, pattern: str) -> list[int]:
    n, m = len(text), len(pattern)
    results = []
    
    for i in range(n - m + 1):
        j = 0
        while j < m:
            if text[i + j] != pattern[j]:
                break
            j += 1
        
        if j == m:
            results.append(i)
            
    return results


: 

In [ ]:
run_test_suite(str_naive, "naive", test_cases)

Testing naive...
  [PASS] Case 0: ('sadbutsad', 'sad') -> [0, 6]
  [PASS] Case 1: ('leetcode', 'leeto') -> []
  [PASS] Case 2: ('aaaaa', 'aa') -> [0, 1, 2, 3]
  [PASS] Case 3: ('abc', '') -> [0, 1, 2, 3]
------------------------------
✅ naive: All test cases passed!




True

: 

### 2. Rabin-Karp (滾動雜湊算法)

#### 演算法講解

Rabin-Karp (簡稱 RK) 演算法提出了一個巧妙的思路：**與其比較字串，不如比較字串的「雜湊值 (Hash)」**。

如果兩個字串的雜湊值不同，那它們一定不是同一個字串。如果雜湊值相同，那它們*很可能*是同一個字串。

它的核心思想是「**滾動雜湊 (Rolling Hash)**」：

1.  **定義雜湊函數**: 我們可以把字串當作一個 $k$ 進位的數字。例如，如果 `pattern="abc"` (假設 `a=1, b=2, c=3`，`BASE=26`)，它的雜湊值可以看作是 $1 \cdot 26^2 + 2 \cdot 26^1 + 3 \cdot 26^0$。為了防止這個數字過大，我們會對一個**大質數 (MOD)** 取餘數。
    * $H(\text{"abc"}) = (H(\text{"ab"}) \cdot \text{BASE} + \text{val}(\text{'c'})) \pmod{\text{MOD}}$

2.  **計算初始雜湊**:
    * 計算 `pattern` 的雜湊值 `pattern_hash` (長度 $M$)。
    * 計算 `text` 中**第一個窗口** (`text[0...M-1]`) 的雜湊值 `window_hash`。

3.  **滑動窗口**:
    * 迴圈 `i` 從 0 到 $N-M$：
    * **比較雜湊**: 比較 `pattern_hash` 和 `window_hash`。
    * **[關鍵] 處理碰撞**: 如果雜湊值**相同**，這可能是一個真正的匹配，也可能只是一個「雜湊碰撞」(Spurious Hit)。我們**必須**再進行一次逐字的字串比較 (`text[i : i+m] == pattern`) 來確認。
    * 如果確認是真匹配，將 `i` 加入 `results` list。
    * **[核心] 滾動雜湊**: 計算下一個窗口 ( `i+1` ) 的雜湊值。我們不需要 $O(M)$ 的時間重新計算，而是可以在 $O(1)$ 的時間內「滾動」：
        1.  **減去** `text[i]` (最左邊字元) 的貢獻。
        2.  將 `window_hash` **乘以** `BASE`。
        3.  **加上** `text[i+m]` (新進入的字元) 的貢獻。
        4.  全部操作都在 `MOD` 下進行。

#### 滾動雜湊的 $O(1)$ 計算
$H(\text{"bcd"}) = ( H(\text{"abc"}) - \text{val}(\text{'a'}) \cdot \text{BASE}^{M-1} ) \cdot \text{BASE} + \text{val}(\text{'d'}) \pmod{\text{MOD}}$

這個 $O(1)$ 的滾動是 RK 演算法平均 $O(N)$ 效能的關鍵。



#### 